In [ ]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from bundle.DataCraft import load_sentence_eeg_prob_data
import torch.nn.functional as F

# Constants
NUM_CLASSES = 36
MODEL_PATH = "../../model/ecd/trained_eegcnn_model_selected_channels.pth"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SELECTED_CHANNELS = [10, 33, 48, 50, 52, 55, 59, 61]

# Dataset class
class EEGDataset(Dataset):
    def __init__(self, data, label_encoder):
        self.data = data
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        chunk = np.array(self.data[idx]["eeg_chunk"], dtype=np.float32)[:, :, SELECTED_CHANNELS]
        chunk[30] *= 3.0
        label = self.label_encoder.transform([self.data[idx]["character"]])[0]
        return torch.tensor(chunk).unsqueeze(0), torch.tensor(label)

# Model Definition
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool3d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _, _ = x.size()
        y = self.pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1, 1)
        return x * y.expand_as(x)

class EEGCNN(nn.Module):
    def __init__(self, num_classes=NUM_CLASSES):
        super(EEGCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(32)

        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        self.se1 = SEBlock(64)

        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm3d(128)
        self.se2 = SEBlock(128)

        self.pool = nn.AdaptiveAvgPool3d(1)
        self.fc1 = nn.Linear(128, 64)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.se1(x)
        x = torch.relu(self.bn3(self.conv3(x)))
        x = self.se2(x)
        x = self.pool(x).squeeze()
        x = self.dropout(torch.relu(self.fc1(x)))
        return self.fc2(x)

# Load data
raw_data = load_sentence_eeg_prob_data()
if not raw_data:
    raise ValueError("Failed to load data.")

all_labels = [item["character"] for item in raw_data]
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Create test set
_ , test_set = train_test_split(raw_data, test_size=0.2, random_state=42)
test_dataset = EEGDataset(test_set, label_encoder)
test_loader = DataLoader(test_dataset, batch_size=8)

# Load model
model = EEGCNN()
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.to(DEVICE)
model.eval()

# Evaluation
correct = total = 0
y_true, y_pred = [], []

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        outputs = model(inputs)
        preds = outputs.argmax(dim=1)
        correct += (preds == targets).sum().item()
        total += targets.size(0)
        y_true.extend(targets.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print(f"\nTest Accuracy: {correct / total:.2%}")
print("\nClassification Report:")
from sklearn.metrics import classification_report

print(classification_report(
    y_true,
    y_pred,
    labels=label_encoder.transform(label_encoder.classes_),
    target_names=label_encoder.classes_,
    zero_division=0
))


Attempting to load processed data from: ../../data/sentences_eeg.pkl
Successfully loaded processed data.
